In [1]:
##check for gpu
!nvidia-smi -L

GPU 0: Tesla K80 (UUID: GPU-f6c6242f-826a-6450-a3d3-f3c0b87492b1)


##Get helper Functions



In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

#import serires of funtions 
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

--2021-08-10 05:59:39--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-08-10 05:59:39 (66.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



##Get a text dataset

link :https://www.kaggle.com/c/nlp-getting-started


In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

#unzip data
unzip_data("nlp_getting_started.zip")

--2021-08-10 05:59:41--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.140.128, 108.177.15.128, 173.194.76.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.140.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.006s  

2021-08-10 05:59:41 (104 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



##Visulaizing the dataset

In [4]:
import pandas as pd
train_df=pd.read_csv("train.csv")
test_df=pd.read_csv("test.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
#shuffle training dataframe
train_df_shuffled =train_df.sample(frac=1,random_state=42)
train_df_shuffled.head()


,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [6]:
#test dataframe look
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [7]:
#count the examples
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [8]:
len(test_df),len(train_df)

(3263, 7613)

In [9]:
#visulalize random samples
import random
random_index=random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
   _,text,target=row
   print(f"Target :{target}","(real disater)" if target > 0 else "(not real disater)")
   print(f"Text :\n {text}\n")
   print("---\n")

Target :0 (not real disater)
Text :
 Magic City Mayhem: Kissimmee adventures ? Aug. 5 2015 http://t.co/FpYrU5GOLh

---

Target :0 (not real disater)
Text :
 Social Casualty #MTVHottest 5SOS

---

Target :1 (real disater)
Text :
 Watch This Airport Get Swallowed Up By A Sandstorm In Under A Minute http://t.co/1tr2KvXCTW

---

Target :0 (not real disater)
Text :
 #LukeBox something about first responders/ military they are our true Hero's!! Besides your music

---

Target :0 (not real disater)
Text :
 BODY BAGS! https://t.co/0McXc68GZD

---



##Split data into train and Validation

In [10]:
from sklearn.model_selection import train_test_split


In [11]:
#use train test split 
train_sentences,val_sentences,train_labels,val_labels=train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                       train_df_shuffled["target"].to_numpy(),
                                                                       test_size=0.1,
                                                                       random_state=42)

In [12]:
#check the length
len(train_sentences),len(val_sentences),len(train_labels),len(val_labels)

(6851, 762, 6851, 762)

In [13]:
train_sentences[:10],train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

##Converting Text into numbers

when dealing with text problem ,

1. builld the model to convert the words into numbers
  
  *Tokanization
  
  *embedding



##Tokenization

In [14]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


In [15]:
#use the defult text vectorization parameters
text_vectorizer =TextVectorization(max_tokens=None,#how many words taken into token
                                   standardize="lower_and_strip_punctuation",
                                   split="whitespace",
                                   ngrams=None,
                                   output_mode="int",
                                   output_sequence_length=None,
                                   pad_to_max_tokens=True)

In [16]:
text_vectorizer

In [17]:
#find the avarge number of tokens
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [18]:
#setup the text vectorization varibales
max_vocab_length=10000 #max number in vocabulary
max_length=15 #max length our sequences 

text_vectorizer =TextVectorization(max_tokens=max_vocab_length,
                                   output_mode="int",
                                   output_sequence_length=max_length)



In [19]:
#fit the text vectorizer to train data
text_vectorizer.adapt(train_sentences)

In [20]:
#craete a sample sentence and tokanize it
sample_sentence ="there is flood in my street"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 74,   9, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
#choose random sentence from the training daatset
random_sentence=random.choice(train_sentences)
print(f"Orignal text :\n {random_sentence} \
      \n\nVectorized version:")

text_vectorizer([random_sentence])

Orignal text :
 The possible new jerseys for the Avalanche next year. ???? http://t.co/nruzhR5XQu       

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[  2, 533,  50,   1,  10,   2, 611, 274, 183,   1,   0,   0,   0,
          0,   0]])>

In [22]:
#get the unique words in the vocabulary
words_in_vocab=text_vectorizer.get_vocabulary() #get all the unique words in train data
top_5_words=words_in_vocab[:5]
bottom_5_words=words_in_vocab[:-5]

print(f"Number of words in vocab :{len(words_in_vocab)}")
print(f"5 most common words in vocab :{top_5_words}")
print(f"5 least common of words in vocab :{bottom_5_words}")

Number of words in vocab :10000
5 most common words in vocab :['', '[UNK]', 'the', 'a', 'in']
5 least common of words in vocab :['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is', 'for', 'on', 'you', 'my', 'with', 'it', 'that', 'at', 'by', 'this', 'from', 'be', 'are', 'was', 'have', 'like', 'as', 'up', 'so', 'just', 'but', 'me', 'im', 'your', 'not', 'amp', 'out', 'its', 'will', 'an', 'no', 'has', 'fire', 'after', 'all', 'when', 'we', 'if', 'now', 'via', 'new', 'more', 'get', 'or', 'about', 'what', 'he', 'people', 'news', 'been', 'over', 'one', 'how', 'dont', 'they', 'who', 'into', 'were', 'do', 'us', '2', 'can', 'video', 'emergency', 'there', 'disaster', 'than', 'police', 'would', 'his', 'still', 'her', 'some', 'body', 'storm', 'crash', 'burning', 'suicide', 'back', 'man', 'california', 'why', 'time', 'them', 'had', 'buildings', 'rt', 'first', 'cant', 'see', 'got', 'day', 'off', 'our', 'going', 'nuclear', 'know', 'world', 'bomb', 'fires', 'love', 'killed', 'go', 'attack', 'yo

##Create a Embedding using Embedding Layer

PARAMETERS:

1.INPUT DIM-size of the vocabulary

2.OUTPUT DIM -size of the output embedding vector

3.INPUT LENGTH-length of the sequence being passed to the embedding layer

In [23]:
from tensorflow.keras import layers
embedding=layers.Embedding(input_dim=max_vocab_length,
                           output_dim=128,
                           input_length=max_length)

embedding

In [24]:
random_sentence=random.choice(train_sentences)
print(f"Orignal Text :\n{random_sentence}\
      \n\nEmbedded Vesrion :")

#embed the setence
sample_embed =embedding(text_vectorizer([random_sentence]))
sample_embed

Orignal Text :
@IAFFLocal4416 no lives lost except those of first responders or the public they're charged with protecting?      

Embedded Vesrion :


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00074438,  0.04370395,  0.04819509, ..., -0.00622781,
         -0.02840428, -0.0350857 ],
        [ 0.04683925,  0.01645425,  0.02689337, ..., -0.04491055,
         -0.02332078,  0.03929896],
        [-0.04289973,  0.01310346,  0.02073182, ..., -0.01123972,
          0.04725409,  0.04728356],
        ...,
        [-0.002971  ,  0.01381961,  0.04319059, ..., -0.00780779,
          0.01763189,  0.0060735 ],
        [-0.04950703, -0.00170275, -0.03439599, ...,  0.03039975,
         -0.01732592, -0.03283159],
        [-0.0491082 ,  0.00875318,  0.03031191, ...,  0.01247232,
         -0.0457153 , -0.01957485]]], dtype=float32)>

In [25]:
#check out single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00074438,  0.04370395,  0.04819509,  0.04609462,  0.02014658,
         0.02886486, -0.03250518,  0.02451554, -0.03662318,  0.03572837,
        -0.02094289, -0.0419922 ,  0.03839404,  0.04350204,  0.02869001,
         0.03450432,  0.0365255 , -0.03595765, -0.00040791,  0.02940524,
        -0.04819323, -0.02420334,  0.04314326, -0.00031688, -0.03413111,
         0.01912243,  0.03076718,  0.0413794 , -0.02939397, -0.04237925,
         0.00826163,  0.03165752,  0.04851455,  0.038965  ,  0.02569209,
         0.02575282,  0.03977991,  0.00840317,  0.01909867, -0.04447844,
        -0.00052434, -0.03496708,  0.00441762,  0.04286372,  0.01086439,
         0.01384943, -0.04673554, -0.01849042, -0.0082323 , -0.04145093,
         0.02572506, -0.03495678,  0.0044903 , -0.04285151,  0.00158916,
         0.03151608, -0.01084648, -0.01591719,  0.02105527,  0.01950467,
         0.02078568, -0.01859761, -0.04837507,  0.04133746, -0.02042637,
  

##Modeling a Text Dataset and Runnning Experiments

model 0-naive bayes(baseline)

model 1-Feed forward neaural Network(dense model)

model 2-LSTM Model

model 3-GRU Model

model 4-Bidirectional Model

model 5-ID Convolutional Neural Network Model

model 6-Tensorflow hub pre trained feature extractor Model

model 7-same model with 10% training data Model

*steps:

+create the model

+compile the model

+fit the model

+evaluate the model


##Model 0 -Getting a Baseline model

In [26]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

#create tokanization and modeling pipeline
model_0=Pipeline([
                  ("tfidf",TfidfVectorizer()),
                  ("clf",MultinomialNB())
])

#fit the pipeline to the training data
model_0.fit(train_sentences,train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
#evaluate the baseline model
baseline_score=model_0.score(val_sentences,val_labels)
print(f"model arcive score :{baseline_score*100:.2f}")
baseline_score

model arcive score :79.27


0.7926509186351706

In [28]:
#make predictions
baseline_preds=model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

##Create evaluation Function

Accuracy

precition

recall

f1-score


In [29]:
#funtion to evaluate
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calcualte_results(y_true,y_preds):
  """
  Calculates model accuracy,preciton,recall and f1 score of a binary classification model
  """

  #calcualte model accuracy
  model_accuracy=accuracy_score(y_true,y_preds)*100

  #calculate model prestion,recall,f1-score 
  model_precision,model_recall,model_f1,_ =precision_recall_fscore_support(y_true,y_preds,average="weighted")
  model_results={"accuracy":model_accuracy,
                 "precision":model_precision,
                 "recall":model_recall,
                 "f1":model_f1}

  return model_results



In [30]:
#get baseline results
baseline_results =calcualte_results(y_true=val_labels,
                                    y_preds=baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Model 1 -Build a Dense Model

In [31]:
#crate a tensorboard callbacks
from helper_functions import create_tensorboard_callback

#create directory to save logs
SAVE_DIR="model_logs"

In [32]:
#build model with functional API
from tensorflow.keras import layers

inputs =layers.Input(shape=(1,),dtype =tf.string)

#turn text into numbers
x=text_vectorizer(inputs)

#create an embedding of the numberized inputs
x=embedding(x)

x=layers.GlobalAveragePooling1D()(x)

outputs=layers.Dense(1,activation="sigmoid")(x)

model_1=tf.keras.Model(inputs,outputs,name="model_1")

In [33]:
model_1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [34]:
#compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [35]:
#fit the model
model_1_history=model_1.fit(x=train_sentences,
                            y=train_labels,
                            epochs=5,
                            validation_data=(val_sentences,val_labels),
                            callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                   experiment_name="model_1_dense")])


Saving TensorBoard log files to: model_logs/model_1_dense/20210810-055951
Epoch 1/5
215/215 [==============================] - 7s 18ms/step - loss: 0.6118 - accuracy: 0.6847 - val_loss: 0.5364 - val_accuracy: 0.7664
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4424 - accuracy: 0.8170 - val_loss: 0.4726 - val_accuracy: 0.7861
Epoch 3/5
215/215 [==============================] - 3s 16ms/step - loss: 0.3484 - accuracy: 0.8549 - val_loss: 0.4687 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2856 - accuracy: 0.8889 - val_loss: 0.4631 - val_accuracy: 0.7887
Epoch 5/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2394 - accuracy: 0.9130 - val_loss: 0.4787 - val_accuracy: 0.7822


In [36]:
#check the results
model_1.evaluate(val_sentences,val_labels)

24/24 [==============================] - 0s 4ms/step - loss: 0.4787 - accuracy: 0.7822


[0.47869673371315, 0.7821522355079651]

In [37]:
#make predictions
model_1_pred_probs=model_1.predict(val_sentences)
model_1_pred_probs

array([[3.88099015e-01],
       [7.96958804e-01],
       [9.97510552e-01],
       [1.15040228e-01],
       [1.14617757e-01],
       [9.42261696e-01],
       [9.17564452e-01],
       [9.92534518e-01],
       [9.59150553e-01],
       [2.77325630e-01],
       [1.26243487e-01],
       [6.87189221e-01],
       [5.32853194e-02],
       [1.75574720e-01],
       [5.44625707e-03],
       [1.39831990e-01],
       [3.33424471e-02],
       [9.18031856e-02],
       [2.44622916e-01],
       [5.24069607e-01],
       [9.13032532e-01],
       [5.55154495e-02],
       [4.37968463e-01],
       [8.59097391e-02],
       [9.59515214e-01],
       [9.98819053e-01],
       [4.12844010e-02],
       [6.03125691e-02],
       [3.05040646e-02],
       [2.07355663e-01],
       [5.79238474e-01],
       [2.58676410e-01],
       [4.89664167e-01],
       [2.31322959e-01],
       [4.82203275e-01],
       [6.35607913e-02],
       [9.94746983e-01],
       [1.50659546e-01],
       [3.68958414e-02],
       [9.98013854e-01],


In [38]:
#convert predicions to label format
model_1_preds=tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [39]:
#calculte model_1_results
model_1_results=calcualte_results(y_true=val_labels,
                                  y_preds=model_1_preds)

model_1_results

{'accuracy': 78.21522309711287,
 'f1': 0.7792361147360404,
 'precision': 0.7864332425219001,
 'recall': 0.7821522309711286}

In [40]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

##Visializing Laerned Embeddings

In [41]:
#get the vocabulary from the text vectorization
words_in_vocab=text_vectorizer.get_vocabulary()
len(words_in_vocab),words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [42]:
#Model 1 summary
model_1.summary()


Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [43]:
#get the weight metrics of embedding layer
embed_weights =model_1.get_layer("embedding").get_weights()[0]
print(embed_weights.shape)

(10000, 128)


In [44]:
embed_weights

array([[-0.05545212,  0.06616012,  0.04643367, ..., -0.03852731,
         0.02921119, -0.06592689],
       [-0.01216213,  0.05564751,  0.05995179, ..., -0.01771172,
        -0.01852385, -0.04701091],
       [ 0.00781463,  0.04156107,  0.01754872, ..., -0.04143303,
        -0.02194763,  0.03087405],
       ...,
       [-0.03446595, -0.0336733 ,  0.04073341, ..., -0.00476253,
        -0.03409411,  0.00487854],
       [ 0.00725148,  0.05713675,  0.08245722, ..., -0.0078408 ,
         0.02576099, -0.07299479],
       [-0.0637591 ,  0.05496721, -0.00529575, ..., -0.08296935,
         0.04797297, -0.03201617]], dtype=float32)

In [45]:
#create embedding files
import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [46]:
#downlaod files from colab to projector
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##Recurrent Neural Network